# **Chapter 6: Stacks and Queues**

> *"Queues are for breadth, stacks are for depth. Choose your traversal wisely."*

---

## **6.1 Introduction**

**Stacks** and **Queues** are fundamental abstract data types (ADTs) that restrict how elements are accessed, providing elegant solutions to a wide range of computational problems. While arrays and linked lists offer general-purpose storage, stacks and queues impose specific access patterns that mirror real-world behaviors:

- **Stacks** model **LIFO** (Last-In-First-Out) behavior: the last element added is the first to be removed (like a stack of plates)
- **Queues** model **FIFO** (First-In-First-Out) behavior: the first element added is the first to be removed (like a line at a ticket counter)

These seemingly simple restrictions make stacks and queues indispensable in algorithm design, operating systems, and software engineering.

---

## **6.2 Stack ADT: Array vs Linked List Implementation**

### **6.2.1 Stack Abstract Data Type**

A **stack** supports three core operations:
- **Push**: Add an element to the top
- **Pop**: Remove and return the top element
- **Peek/Top**: View the top element without removing it

```
┌─────────────────────────────────────────────────────────────────────┐
│                    STACK OPERATIONS VISUALIZATION                    │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Initial:  Empty                                                     │
│                                                                      │
│  push(10)          push(20)          push(30)         pop()         │
│                                                                      │
│     │                  │                  │               │          │
│     ▼                  ▼                  ▼               ▼          │
│  ┌─────┐            ┌─────┐            ┌─────┐         ┌─────┐     │
│  │ 10  │            │ 20  │            │ 30  │         │ 20  │     │
│  └─────┘            ├─────┤            ├─────┤         ├─────┤     │
│  TOP                │ 10  │            │ 20  │         │ 10  │     │
│                     └─────┘            ├─────┤         └─────┘     │
│  TOP                TOP                │ 10  │         TOP         │
│                                        └─────┘                       │
│                                        TOP                           │
│                                                                      │
│  Result: Returns 30                                                  │
│                                                                      │
│  Stack Principle: LIFO (Last-In-First-Out)                          │
│  The last element pushed (30) is the first one popped               │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

### **6.2.2 Array-Based Stack Implementation**

Arrays provide cache-friendly stack implementation with O(1) operations.

```python
from typing import TypeVar, Generic, Optional, Iterator

T = TypeVar('T')

class ArrayStack(Generic[T]):
    """
    Stack implementation using a dynamic array.
    
    Time Complexities:
        push: O(1) amortized (O(n) when resizing)
        pop: O(1) amortized (O(n) when shrinking)
        peek: O(1)
        is_empty: O(1)
    
    Space Complexity: O(n) where n is number of elements
    
    This is the preferred implementation when:
        - Maximum capacity is known or can be estimated
        - Cache performance is important
        - Memory overhead should be minimal
    """
    
    def __init__(self, initial_capacity: int = 10):
        """
        Initialize empty stack.
        
        Args:
            initial_capacity: Starting array size (grows dynamically)
        """
        self._capacity = initial_capacity
        self._data: list[Optional[T]] = [None] * initial_capacity
        self._size = 0  # Also serves as index of next free slot (top)
    
    def push(self, item: T) -> None:
        """
        Add item to top of stack.
        
        Time: O(1) amortized
        """
        # Check if resize needed
        if self._size == self._capacity:
            self._resize(2 * self._capacity)
        
        self._data[self._size] = item
        self._size += 1
    
    def pop(self) -> T:
        """
        Remove and return top item.
        
        Time: O(1) amortized
        
        Raises:
            IndexError: If stack is empty
        """
        if self.is_empty():
            raise IndexError("pop from empty stack")
        
        self._size -= 1
        item = self._data[self._size]
        self._data[self._size] = None  # Help garbage collection
        
        # Shrink if utilization drops below 25%
        if 0 < self._size < self._capacity // 4:
            self._resize(self._capacity // 2)
        
        return item
    
    def peek(self) -> T:
        """
        Return top item without removing it.
        
        Time: O(1)
        """
        if self.is_empty():
            raise IndexError("peek from empty stack")
        return self._data[self._size - 1]
    
    def is_empty(self) -> bool:
        """Check if stack is empty."""
        return self._size == 0
    
    def __len__(self) -> int:
        """Return number of elements."""
        return self._size
    
    def _resize(self, new_capacity: int) -> None:
        """
        Resize internal array.
        
        Time: O(n)
        """
        new_data = [None] * new_capacity
        for i in range(self._size):
            new_data[i] = self._data[i]
        self._data = new_data
        self._capacity = new_capacity
    
    def __iter__(self) -> Iterator[T]:
        """
        Iterate from top to bottom.
        
        Note: This exposes internal order, useful for debugging
        but not part of standard Stack ADT.
        """
        for i in range(self._size - 1, -1, -1):
            yield self._data[i]
    
    def __str__(self) -> str:
        if self.is_empty():
            return "Empty Stack"
        elements = [str(self._data[i]) for i in range(self._size)]
        return "Top -> " + " -> ".join(reversed(elements))


def demonstrate_array_stack():
    """
    Demonstrate array-based stack operations.
    """
    print("Array-Based Stack Implementation")
    print("=" * 70)
    
    stack = ArrayStack[int](initial_capacity=4)
    
    print("Pushing: 10, 20, 30, 40, 50")
    for val in [10, 20, 30, 40, 50]:
        stack.push(val)
        print(f"  Pushed {val}, size={len(stack)}")
    
    print(f"\nStack state: {stack}")
    print(f"Top element (peek): {stack.peek()}")
    
    print("\nPopping elements:")
    while not stack.is_empty():
        val = stack.pop()
        print(f"  Popped {val}, remaining={len(stack)}")
    
    print("""
    
    Array Stack Analysis:
    ─────────────────────────────────────────────────────────────────────
    
    Advantages:
      ✓ Excellent cache locality (contiguous memory)
      ✓ No pointer overhead per element
      ✓ Fast access to top element (array index)
      ✓ Memory efficient
    
    Disadvantages:
      ✗ Fixed maximum capacity (unless dynamic resizing used)
      ✗ Resizing cost when capacity exceeded
      ✗ Potential memory waste if capacity >> size
    
    Use Cases:
      • Expression evaluation (compiler/parser)
      • Function call stack (runtime implementation)
      • Undo/Redo operations
      • Depth-First Search (DFS)
    """)


demonstrate_array_stack()
```

**Output:**
```
Array-Based Stack Implementation
======================================================================
Pushing: 10, 20, 30, 40, 50
  Pushed 10, size=1
  Pushed 20, size=2
  Pushed 30, size=3
  Pushed 40, size=4
  Pushed 50, size=5

Stack state: Top -> 50 -> 40 -> 30 -> 20 -> 10
Top element (peek): 50

Popping elements:
  Popped 50, remaining=4
  Popped 40, remaining=3
  Popped 30, remaining=2
  Popped 20, remaining=1
  Popped 10, remaining=0


Array Stack Analysis:
─────────────────────────────────────────────────────────────────────

Advantages:
  ✓ Excellent cache locality (contiguous memory)
  ✓ No pointer overhead per element
  ✓ Fast access to top element (array index)
  ✓ Memory efficient

Disadvantages:
  ✗ Fixed maximum capacity (unless dynamic resizing used)
  ✗ Resizing cost when capacity exceeded
  ✗ Potential memory waste if capacity >> size

Use Cases:
  • Expression evaluation (compiler/parser)
  • Function call stack (runtime implementation)
  • Undo/Redo operations
  • Depth-First Search (DFS)
```

---

### **6.2.3 Linked List-Based Stack Implementation**

```python
class LinkedStack(Generic[T]):
    """
    Stack implementation using a singly linked list.
    
    Time Complexities:
        push: O(1)
        pop: O(1)
        peek: O(1)
        is_empty: O(1)
    
    Space Complexity: O(n)
    
    This is preferred when:
        - Stack size is highly variable
        - Memory must be allocated on demand
        - No estimate of maximum size available
    """
    
    class _Node(Generic[T]):
        __slots__ = ['data', 'next']
        
        def __init__(self, data: T):
            self.data = data
            self.next: Optional['LinkedStack._Node[T]'] = None
    
    def __init__(self):
        self._top: Optional[LinkedStack._Node[T]] = None
        self._size = 0
    
    def push(self, item: T) -> None:
        """
        Push item onto stack.
        
        Time: O(1)
        
        Algorithm:
        1. Create new node
        2. Point new node to current top
        3. Update top to new node
        """
        new_node = self._Node(item)
        new_node.next = self._top
        self._top = new_node
        self._size += 1
    
    def pop(self) -> T:
        """
        Pop item from stack.
        
        Time: O(1)
        """
        if self.is_empty():
            raise IndexError("pop from empty stack")
        
        data = self._top.data
        self._top = self._top.next
        self._size -= 1
        return data
    
    def peek(self) -> T:
        """Return top element without removing."""
        if self.is_empty():
            raise IndexError("peek from empty stack")
        return self._top.data
    
    def is_empty(self) -> bool:
        return self._top is None
    
    def __len__(self) -> int:
        return self._size
    
    def __str__(self) -> str:
        if self.is_empty():
            return "Empty Stack"
        
        elements = []
        current = self._top
        while current:
            elements.append(str(current.data))
            current = current.next
        
        return "Top -> " + " -> ".join(elements)


def compare_implementations():
    """
    Compare array and linked list stack implementations.
    """
    print("Comparison: Array vs Linked List Stack")
    print("=" * 70)
    
    comparison = """
    ┌─────────────────────┬────────────────────────┬────────────────────────┐
    │      Aspect         │     Array Stack        │   Linked List Stack    │
    ├─────────────────────┼────────────────────────┼────────────────────────┤
    │ push                │ O(1) amortized         │ O(1)                   │
    │ pop                 │ O(1) amortized         │ O(1)                   │
    │ peek                │ O(1)                   │ O(1)                   │
    │ Memory allocation   │ Contiguous (cache-friendly) │ Scattered        │
    │ Memory overhead     │ Low (just array)       │ High (1 pointer/node)  │
    │ Resizing            │ Required occasionally  │ Never needed           │
    │ Memory usage        │ May waste capacity     │ Exact allocation       │
    │ Implementation      │ Simpler                │ Slightly more complex  │
    └─────────────────────┴────────────────────────┴────────────────────────┘
    
    Industry Standard:
      • C++ std::stack: Typically uses std::deque (hybrid) by default
      • Java Stack: Extends Vector (dynamic array)
      • Python list: Used as stack (append/pop from end)
      • C# Stack: Array-based implementation
    
    Recommendation:
      • Use ArrayStack for most applications (better cache performance)
      • Use LinkedStack only when memory fragmentation is acceptable
        and stack size is extremely unpredictable
    """
    
    print(comparison)


compare_implementations()
```

---

## **6.3 Applications of Stacks**

### **6.3.1 Expression Evaluation (Infix to Postfix)**

Stacks are essential for parsing and evaluating mathematical expressions.

```python
def expression_evaluation():
    """
    Evaluate arithmetic expressions using stacks.
    """
    
    print("Expression Evaluation Using Stacks")
    print("=" * 70)
    
    print("""
    Notation Types:
    ─────────────────────────────────────────────────────────────────────
    
    Infix:   Operator between operands (human-readable)
             Example: 3 + 4 * 2 / (1 - 5)
    
    Prefix:  Operator before operands (Polish notation)
             Example: + 3 / * 4 2 - 1 5
    
    Postfix: Operator after operands (Reverse Polish notation)
             Example: 3 4 2 * 1 5 - / +
    
    Why Postfix?
      • No parentheses needed (implied by order)
      • Easy to evaluate using a single stack
      • Unambiguous parsing
    """)
    
    def infix_to_postfix(expression: str) -> str:
        """
        Convert infix expression to postfix using Shunting Yard algorithm.
        
        Time: O(n)
        Space: O(n)
        """
        # Operator precedence
        precedence = {'+': 1, '-': 1, '*': 2, '/': 2, '^': 3}
        
        stack = []  # Operator stack
        output = []  # Output list
        
        i = 0
        while i < len(expression):
            char = expression[i]
            
            # Skip whitespace
            if char.isspace():
                i += 1
                continue
            
            # Operand (number)
            if char.isdigit():
                num = char
                while i + 1 < len(expression) and expression[i + 1].isdigit():
                    i += 1
                    num += expression[i]
                output.append(num)
            
            # Left parenthesis
            elif char == '(':
                stack.append(char)
            
            # Right parenthesis
            elif char == ')':
                while stack and stack[-1] != '(':
                    output.append(stack.pop())
                stack.pop()  # Remove '('
            
            # Operator
            else:
                while (stack and stack[-1] != '(' and
                       precedence.get(stack[-1], 0) >= precedence.get(char, 0)):
                    output.append(stack.pop())
                stack.append(char)
            
            i += 1
        
        # Pop remaining operators
        while stack:
            output.append(stack.pop())
        
        return ' '.join(output)
    
    def evaluate_postfix(expression: str) -> float:
        """
        Evaluate postfix expression using stack.
        
        Time: O(n)
        Space: O(n)
        """
        stack = []
        
        for token in expression.split():
            if token.isdigit():
                stack.append(int(token))
            else:
                # Operator: pop two operands
                b = stack.pop()
                a = stack.pop()
                
                if token == '+':
                    stack.append(a + b)
                elif token == '-':
                    stack.append(a - b)
                elif token == '*':
                    stack.append(a * b)
                elif token == '/':
                    stack.append(a / b)
                elif token == '^':
                    stack.append(a ** b)
        
        return stack[0]
    
    # Demonstration
    infix = "3 + 4 * 2 / ( 1 - 5 )"
    print(f"Infix expression: {infix}")
    
    postfix = infix_to_postfix(infix)
    print(f"Postfix: {postfix}")
    
    result = evaluate_postfix(postfix)
    print(f"Result: {result}")
    
    # Step by step
    print("\nStep-by-step evaluation:")
    print("-" * 40)
    print("Token    Stack")
    stack = []
    for token in postfix.split():
        if token.isdigit():
            stack.append(int(token))
            print(f"{token:<8} {stack}")
        else:
            b = stack.pop()
            a = stack.pop()
            if token == '+':
                res = a + b
            elif token == '-':
                res = a - b
            elif token == '*':
                res = a * b
            elif token == '/':
                res = a / b
            stack.append(res)
            print(f"{token:<8} {stack}  (applied {a} {token} {b})")
    
    print("""
    
    Algorithm Complexity:
    ─────────────────────────────────────────────────────────────────────
    
    Infix to Postfix (Shunting Yard):
      Time: O(n) - single pass through expression
      Space: O(n) - for operator stack and output
    
    Postfix Evaluation:
      Time: O(n) - single pass through tokens
      Space: O(n) - for operand stack
    
    Real-world Applications:
      • Calculator implementations
      • Spreadsheet formula evaluation
      • Compiler expression parsing
      • SQL query optimizers
    """)


expression_evaluation()
```

**Output:**
```
Expression Evaluation Using Stacks
======================================================================

Notation Types:
─────────────────────────────────────────────────────────────────────

Infix:   Operator between operands (human-readable)
         Example: 3 + 4 * 2 / (1 - 5)

Prefix:  Operator before operands (Polish notation)
         Example: + 3 / * 4 2 - 1 5

Postfix: Operator after operands (Reverse Polish notation)
         Example: 3 4 2 * 1 5 - / +

Why Postfix?
  • No parentheses needed (implied by order)
  • Easy to evaluate using a single stack
  • Unambiguous parsing


Infix expression: 3 + 4 * 2 / ( 1 - 5 )
Postfix: 3 4 2 * 1 5 - / +
Result: 1.0

Step-by-step evaluation:
----------------------------------------
Token    Stack
3        [3]
4        [3, 4]
2        [3, 4, 2]
*        [3, 8]  (applied 4 * 2)
1        [3, 8, 1]
5        [3, 8, 1, 5]
-        [3, 8, -4]  (applied 1 - 5)
/        [3, -2.0]  (applied 8 / -4)
+        [1.0]  (applied 3 + -2.0)


Algorithm Complexity:
─────────────────────────────────────────────────────────────────────

Infix to Postfix (Shunting Yard):
  Time: O(n) - single pass through expression
  Space: O(n) - for operator stack and output

Postfix Evaluation:
  Time: O(n) - single pass through tokens
  Space: O(n) - for operand stack

Real-world Applications:
  • Calculator implementations
  • Spreadsheet formula evaluation
  • Compiler expression parsing
  • SQL query optimizers
```

---

### **6.3.2 Parentheses Matching**

```python
def parentheses_matching():
    """
    Check for balanced parentheses using stacks.
    """
    
    print("Parentheses Matching Algorithm")
    print("=" * 70)
    
    def is_balanced(expression: str) -> bool:
        """
        Check if parentheses are balanced.
        
        Time: O(n)
        Space: O(n) in worst case (all opening brackets)
        """
        stack = []
        # Mapping of closing to opening brackets
        pairs = {')': '(', '}': '{', ']': '['}
        
        for char in expression:
            if char in '({[':
                stack.append(char)
            elif char in ')}]':
                if not stack:
                    return False  # Closing bracket without opening
                if stack[-1] != pairs[char]:
                    return False  # Mismatched bracket type
                stack.pop()
        
        return len(stack) == 0  # True if all brackets matched
    
    # Test cases
    test_cases = [
        ("()", True),
        ("()[]{}", True),
        ("([{}])", True),
        ("(]", False),
        ("([)]", False),
        ("{[(])}", False),
        ("((()))", True),
        ("(((", False),
        ("))", False),
    ]
    
    print("Test Cases:")
    print("-" * 50)
    print(f"{'Expression':<20} {'Expected':<10} {'Result':<10} {'Status'}")
    print("-" * 50)
    
    for expr, expected in test_cases:
        result = is_balanced(expr)
        status = "✓" if result == expected else "✗"
        print(f"{expr:<20} {str(expected):<10} {str(result):<10} {status}")
    
    print("""
    
    Algorithm Explanation:
    ─────────────────────────────────────────────────────────────────────
    
    1. Scan expression left to right
    2. If opening bracket: push to stack
    3. If closing bracket:
       a. Check if stack is empty (unmatched closing)
       b. Check if top matches expected opening type
       c. Pop if matched
    4. After scan: stack must be empty (no unmatched opening)
    
    Why Stack?
      • Last opened bracket must be closed first (LIFO)
      • Matches the nesting structure of parentheses
    
    Extensions:
      • Can track position of mismatch for error reporting
      • Can handle multiple bracket types simultaneously
      • Can be extended to check HTML/XML tag nesting
    """)


parentheses_matching()
```

---

### **6.3.3 Undo Operations**

```python
def undo_operations():
    """
    Implement undo functionality using stacks.
    """
    
    print("Undo/Redo Operations Using Stacks")
    print("=" * 70)
    
    class TextEditor:
        """
        Simple text editor with undo/redo functionality.
        
        Uses two stacks:
          undo_stack: Stores previous states for undo
          redo_stack: Stores undone states for redo
        """
        
        def __init__(self):
            self.content = ""
            self.undo_stack = []  # Stores previous states
            self.redo_stack = []  # Stores states that were undone
        
        def type_text(self, text: str) -> None:
            """
            Add text to document.
            
            Clears redo stack (new action invalidates future).
            """
            # Save current state for undo
            self.undo_stack.append(self.content)
            self.content += text
            # Clear redo stack (branching timeline)
            self.redo_stack.clear()
            print(f"Typed: '{text}' -> Document: '{self.content}'")
        
        def delete_text(self, length: int) -> None:
            """Delete last 'length' characters."""
            if length > 0 and self.content:
                self.undo_stack.append(self.content)
                deleted = self.content[-length:]
                self.content = self.content[:-length]
                self.redo_stack.clear()
                print(f"Deleted: '{deleted}' -> Document: '{self.content}'")
        
        def undo(self) -> bool:
            """
            Undo last operation.
            
            Moves current state to redo stack.
            Restores previous state from undo stack.
            """
            if not self.undo_stack:
                print("Nothing to undo")
                return False
            
            # Save current to redo
            self.redo_stack.append(self.content)
            # Restore previous
            self.content = self.undo_stack.pop()
            print(f"Undo -> Document: '{self.content}'")
            return True
        
        def redo(self) -> bool:
            """Redo previously undone operation."""
            if not self.redo_stack:
                print("Nothing to redo")
                return False
            
            self.undo_stack.append(self.content)
            self.content = self.redo_stack.pop()
            print(f"Redo -> Document: '{self.content}'")
            return True
        
        def get_content(self) -> str:
            return self.content
    
    # Demonstration
    editor = TextEditor()
    
    print("Operations:")
    print("-" * 50)
    
    editor.type_text("Hello")
    editor.type_text(" World")
    editor.delete_text(6)  # Delete " World"
    editor.undo()  # Restore " World"
    editor.undo()  # Restore ""
    editor.redo()  # Redo "Hello"
    editor.type_text("!")
    
    print(f"\nFinal content: '{editor.get_content()}'")
    
    print("""
    
    Multi-Level Undo Architecture:
    ─────────────────────────────────────────────────────────────────────
    
    State Management:
      • Each command implements execute() and undo() methods
      • Command pattern encapsulates operations
      • Stacks store command objects or state snapshots
    
    Memory Optimization:
      • Store deltas (changes) rather than full states
      • Limit undo history size (e.g., last 50 operations)
      • Compress consecutive similar operations
    
    Applications:
      • Text editors (vim, VS Code, Word)
      • Graphic design software (Photoshop, Figma)
      • Database transactions (rollback)
      • Version control systems (git stash)
    """)


undo_operations()
```

---

## **6.4 Queue ADT: Linear, Circular, and Priority Implementations**

### **6.4.1 Queue Abstract Data Type**

A **queue** supports:
- **Enqueue**: Add element to the rear
- **Dequeue**: Remove element from the front
- **Front/Peek**: View front element

```
┌─────────────────────────────────────────────────────────────────────┐
│                    QUEUE OPERATIONS VISUALIZATION                    │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Initial:  Empty                                                     │
│                                                                      │
│  enqueue(10)       enqueue(20)       enqueue(30)      dequeue()     │
│                                                                      │
│     │                  │                  │               │          │
│     ▼                  ▼                  ▼               ▼          │
│  ┌─────┐            ┌─────┐            ┌─────┐         ┌─────┐     │
│  │ 10  │            │ 10  │            │ 10  │         │ 20  │     │
│  └─────┘            ├─────┤            ├─────┤         ├─────┤     │
│  FRONT/REAR         │ 20  │            │ 20  │         │ 30  │     │
│                     └─────┘            ├─────┤         └─────┘     │
│                     FRONT   REAR       │ 30  │         FRONT  REAR │
│                                        └─────┘                       │
│                                        FRONT         REAR            │
│                                                                      │
│  Result: Returns 10                                                  │
│                                                                      │
│  Queue Principle: FIFO (First-In-First-Out)                         │
│  The first element enqueued (10) is the first one dequeued          │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

### **6.4.2 Linear Queue (Array Implementation)**

```python
class ArrayQueue(Generic[T]):
    """
    Queue implementation using circular array.
    
    Uses circular buffer technique to avoid shifting elements.
    front and rear pointers wrap around using modulo arithmetic.
    
    Time Complexities:
        enqueue: O(1) amortized
        dequeue: O(1) amortized
        front: O(1)
    
    Space: O(n)
    """
    
    def __init__(self, capacity: int = 10):
        self._capacity = capacity
        self._data: list[Optional[T]] = [None] * capacity
        self._front = 0  # Index of front element
        self._rear = 0   # Index where next element will be inserted
        self._size = 0   # Current number of elements
    
    def enqueue(self, item: T) -> None:
        """
        Add item to rear of queue.
        
        Time: O(1) amortized
        """
        if self._size == self._capacity:
            self._resize(2 * self._capacity)
        
        self._data[self._rear] = item
        self._rear = (self._rear + 1) % self._capacity
        self._size += 1
    
    def dequeue(self) -> T:
        """
        Remove and return front item.
        
        Time: O(1) amortized
        """
        if self.is_empty():
            raise IndexError("dequeue from empty queue")
        
        item = self._data[self._front]
        self._data[self._front] = None  # Help GC
        self._front = (self._front + 1) % self._capacity
        self._size -= 1
        
        if 0 < self._size < self._capacity // 4:
            self._resize(self._capacity // 2)
        
        return item
    
    def front(self) -> T:
        """View front item without removing."""
        if self.is_empty():
            raise IndexError("front from empty queue")
        return self._data[self._front]
    
    def is_empty(self) -> bool:
        return self._size == 0
    
    def __len__(self) -> int:
        return self._size
    
    def _resize(self, new_capacity: int) -> None:
        """Resize internal array."""
        new_data = [None] * new_capacity
        
        # Copy elements in order
        for i in range(self._size):
            new_data[i] = self._data[(self._front + i) % self._capacity]
        
        self._data = new_data
        self._front = 0
        self._rear = self._size
        self._capacity = new_capacity


def demonstrate_queue():
    """
    Demonstrate queue operations.
    """
    print("Circular Array Queue Implementation")
    print("=" * 70)
    
    queue = ArrayQueue[int](capacity=4)
    
    print("Enqueuing: 10, 20, 30, 40")
    for val in [10, 20, 30, 40]:
        queue.enqueue(val)
    
    print(f"Front element: {queue.front()}")
    print(f"Queue size: {len(queue)}")
    
    print("\nDequeuing twice:")
    print(f"  Dequeued: {queue.dequeue()}")
    print(f"  Dequeued: {queue.dequeue()}")
    print(f"  Remaining size: {len(queue)}")
    
    print("\nEnqueuing 50, 60 (demonstrating circular nature):")
    queue.enqueue(50)
    queue.enqueue(60)
    print(f"  Queue size: {len(queue)}")
    
    print("\nEmptying queue:")
    while not queue.is_empty():
        print(f"  Dequeued: {queue.dequeue()}")


demonstrate_queue()
```

---

### **6.4.3 Priority Queue**

A **priority queue** extends the queue ADT by removing elements based on priority (highest or lowest) rather than insertion order.

```python
import heapq

class PriorityQueue(Generic[T]):
    """
    Priority Queue implementation using binary heap.
    
    By default: Min-heap (smallest element has highest priority)
    For max-heap: Store negatives or use custom comparator
    
    Time Complexities:
        insert: O(log n)
        extract_min/max: O(log n)
        peek: O(1)
    
    Space: O(n)
    """
    
    def __init__(self):
        self._heap = []
        self._index = 0  # For stable sorting (tie-breaker)
    
    def push(self, item: T, priority: int = 0) -> None:
        """
        Add item with given priority.
        
        Lower priority number = higher priority (for min-heap)
        """
        # Heapq is min-heap, so we store (priority, index, item)
        # Index ensures stable ordering for equal priorities
        heapq.heappush(self._heap, (priority, self._index, item))
        self._index += 1
    
    def pop(self) -> T:
        """
        Remove and return item with highest priority (lowest number).
        """
        if self.is_empty():
            raise IndexError("pop from empty priority queue")
        return heapq.heappop(self._heap)[2]
    
    def peek(self) -> T:
        """View highest priority item without removing."""
        if self.is_empty():
            raise IndexError("peek from empty priority queue")
        return self._heap[0][2]
    
    def is_empty(self) -> bool:
        return len(self._heap) == 0
    
    def __len__(self) -> int:
        return len(self._heap)


def demonstrate_priority_queue():
    """
    Demonstrate priority queue usage.
    """
    print("Priority Queue Demonstration")
    print("=" * 70)
    
    pq = PriorityQueue[str]()
    
    # Tasks with priorities (lower number = higher priority)
    tasks = [
        ("Send email", 3),
        ("Fix critical bug", 1),
        ("Update documentation", 5),
        ("Code review", 2),
        ("Team meeting", 4),
    ]
    
    print("Adding tasks with priorities:")
    for task, priority in tasks:
        pq.push(task, priority)
        print(f"  [{priority}] {task}")
    
    print("\nProcessing by priority:")
    while not pq.is_empty():
        task = pq.pop()
        print(f"  Doing: {task}")


demonstrate_priority_queue()
```

---

## **6.5 Deque (Double-Ended Queue)**

### **6.5.1 Deque ADT**

A **deque** (double-ended queue) allows insertion and deletion at both ends, combining stack and queue capabilities.

```python
class Deque(Generic[T]):
    """
    Double-ended queue implementation using circular array.
    
    Supports O(1) operations at both ends.
    
    Operations:
        push_front: Add to front
        push_back: Add to back (equivalent to enqueue)
        pop_front: Remove from front (equivalent to dequeue)
        pop_back: Remove from back (equivalent to stack pop)
    """
    
    def __init__(self, capacity: int = 10):
        self._capacity = capacity
        self._data: list[Optional[T]] = [None] * capacity
        self._front = 0
        self._back = 0  # Points to next available slot at back
        self._size = 0
    
    def push_front(self, item: T) -> None:
        """Add item to front."""
        if self._size == self._capacity:
            self._resize(2 * self._capacity)
        
        # Move front back and insert
        self._front = (self._front - 1) % self._capacity
        self._data[self._front] = item
        self._size += 1
    
    def push_back(self, item: T) -> None:
        """Add item to back."""
        if self._size == self._capacity:
            self._resize(2 * self._capacity)
        
        self._data[self._back] = item
        self._back = (self._back + 1) % self._capacity
        self._size += 1
    
    def pop_front(self) -> T:
        """Remove from front."""
        if self.is_empty():
            raise IndexError("pop from empty deque")
        
        item = self._data[self._front]
        self._data[self._front] = None
        self._front = (self._front + 1) % self._capacity
        self._size -= 1
        return item
    
    def pop_back(self) -> T:
        """Remove from back."""
        if self.is_empty():
            raise IndexError("pop from empty deque")
        
        self._back = (self._back - 1) % self._capacity
        item = self._data[self._back]
        self._data[self._back] = None
        self._size -= 1
        return item
    
    def peek_front(self) -> T:
        """View front item."""
        if self.is_empty():
            raise IndexError("peek from empty deque")
        return self._data[self._front]
    
    def peek_back(self) -> T:
        """View back item."""
        if self.is_empty():
            raise IndexError("peek from empty deque")
        return self._data[(self._back - 1) % self._capacity]
    
    def is_empty(self) -> bool:
        return self._size == 0
    
    def __len__(self) -> int:
        return self._size
    
    def _resize(self, new_capacity: int) -> None:
        """Resize array."""
        new_data = [None] * new_capacity
        for i in range(self._size):
            new_data[i] = self._data[(self._front + i) % self._capacity]
        self._data = new_data
        self._front = 0
        self._back = self._size
        self._capacity = new_capacity


def demonstrate_deque():
    """
    Demonstrate deque as both stack and queue.
    """
    print("Double-Ended Queue (Deque) Demonstration")
    print("=" * 70)
    
    deque = Deque[int]()
    
    print("Using as Stack (LIFO) - push_back/pop_back:")
    for i in [1, 2, 3]:
        deque.push_back(i)
    while not deque.is_empty():
        print(f"  {deque.pop_back()}", end=" ")
    print()
    
    print("\nUsing as Queue (FIFO) - push_back/pop_front:")
    for i in [1, 2, 3]:
        deque.push_back(i)
    while not deque.is_empty():
        print(f"  {deque.pop_front()}", end=" ")
    print()
    
    print("\nUsing both ends:")
    deque.push_front(1)  # [1]
    deque.push_back(2)   # [1, 2]
    deque.push_front(0)  # [0, 1, 2]
    deque.push_back(3)   # [0, 1, 2, 3]
    
    print(f"Front: {deque.peek_front()}, Back: {deque.peek_back()}")
    print(f"Pop front: {deque.pop_front()}")  # 0
    print(f"Pop back: {deque.pop_back()}")    # 3
    print(f"Remaining: {deque.peek_front()} to {deque.peek_back()}")


demonstrate_deque()
```

---

## **6.6 Monotonic Stacks and Queues**

### **6.6.1 Monotonic Stack Pattern**

A **monotonic stack** maintains elements in strictly increasing or decreasing order, useful for finding next greater/smaller elements.

```python
def monotonic_stack_patterns():
    """
    Demonstrate monotonic stack patterns for solving range queries.
    """
    
    print("Monotonic Stack: Next Greater Element")
    print("=" * 70)
    
    def next_greater_element(arr):
        """
        Find next greater element for each position.
        
        For each element, find the first element to its right that is larger.
        If none exists, return -1.
        
        Time: O(n) - each element pushed and popped once
        Space: O(n)
        """
        n = len(arr)
        result = [-1] * n
        stack = []  # Stores indices, maintains decreasing order
        
        for i in range(n):
            # While current element is greater than stack top,
            # it is the next greater element for stack top
            while stack and arr[i] > arr[stack[-1]]:
                idx = stack.pop()
                result[idx] = arr[i]
            
            stack.append(i)
        
        return result
    
    # Example
    arr = [4, 5, 2, 25, 7, 18]
    result = next_greater_element(arr)
    
    print(f"Array: {arr}")
    print(f"Next Greater: {result}")
    
    print("\nStep-by-step:")
    stack = []
    for i, val in enumerate(arr):
        print(f"i={i}, val={val}, stack={stack}")
        while stack and val > arr[stack[-1]]:
            idx = stack.pop()
            print(f"  -> {arr[idx]}'s next greater is {val}")
        stack.append(i)
    
    print("""
    
    Monotonic Stack Applications:
    ─────────────────────────────────────────────────────────────────────
    
    1. Next Greater/Smaller Element
       • Finding spans in stock prices
       • Nearest greater to left/right
    
    2. Largest Rectangle in Histogram
       • Maintain increasing stack of bar heights
       • Calculate area when encountering smaller bar
    
    3. Trapping Rain Water
       • Decreasing stack to find boundaries
    
    4. Remove K Digits to Form Smallest Number
       • Monotonic increasing stack
    
    Pattern Template:
    ─────────────────────────────────────────────────────────────────────
    
    stack = []
    for i, x in enumerate(arr):
        # For next greater: while stack and x > arr[stack[-1]]
        # For next smaller: while stack and x < arr[stack[-1]]
        while stack and condition(x, arr[stack[-1]]):
            idx = stack.pop()
            # Process element at idx
            result[idx] = ...
        stack.append(i)
    """)


monotonic_stack_patterns()
```

---

### **6.6.2 Sliding Window Maximum (Monotonic Queue)**

```python
def sliding_window_maximum():
    """
    Find maximum in each sliding window of size k.
    """
    
    print("Sliding Window Maximum using Monotonic Deque")
    print("=" * 70)
    
    def max_sliding_window(nums, k):
        """
        Find maximum in each window of size k.
        
        Time: O(n) - each element added and removed once
        Space: O(k) - deque stores at most k indices
        
        Algorithm:
        1. Maintain deque of indices with decreasing values
        2. Remove elements outside current window
        3. Remove elements smaller than current (they can't be max)
        4. Add current index
        5. Window max is at front of deque
        """
        from collections import deque
        
        result = []
        dq = deque()  # Stores indices, decreasing order of values
        
        for i, num in enumerate(nums):
            # Remove indices out of current window
            while dq and dq[0] < i - k + 1:
                dq.popleft()
            
            # Remove indices with values less than current
            # (they can't be the maximum)
            while dq and nums[dq[-1]] < num:
                dq.pop()
            
            dq.append(i)
            
            # Add to result once window is full
            if i >= k - 1:
                result.append(nums[dq[0]])
        
        return result
    
    nums = [1, 3, -1, -3, 5, 3, 6, 7]
    k = 3
    
    print(f"Array: {nums}")
    print(f"Window size: {k}")
    print(f"Maximums: {max_sliding_window(nums, k)}")
    
    print("\nWindow breakdown:")
    for i in range(len(nums) - k + 1):
        window = nums[i:i+k]
        print(f"  {window} -> max = {max(window)}")
    
    print("""
    
    Why Monotonic Deque Works:
    ─────────────────────────────────────────────────────────────────────
    
    Key Insight:
      If we see a new element x, any previous element smaller than x
      will never be the maximum in any window containing x.
      
    Example:
      Window [3, -1, -3], new element 5 arrives
      3, -1, -3 are all smaller than 5, so we discard them
      New window max is definitely 5
    
    Complexity Analysis:
      • Each element is pushed once and popped once
      • Total operations: O(n)
      • Brute force would be O(n × k)
    
    Applications:
      • Real-time stream processing
      • Moving averages in financial data
      • Maximum in subarrays
    """)


sliding_window_maximum()
```

---

## **6.7 Summary and Key Takeaways**

### **6.7.1 Stack vs Queue Comparison**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    STACK vs QUEUE COMPARISON                         │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  Feature           │ Stack (LIFO)          │ Queue (FIFO)            │
│  ──────────────────┼───────────────────────┼─────────────────────────│
│  Access Pattern    │ Last-In-First-Out     │ First-In-First-Out      │
│  Main Operations   │ push, pop             │ enqueue, dequeue        │
│  Use Case          │ Undo, DFS, Parsing    │ BFS, Scheduling, Buffer │
│  Implementation    │ Array/Linked List     │ Circular Array/Linked   │
│  Peek Operation    │ View top              │ View front              │
│                                                                      │
│  Deque: Combines both - O(1) at both ends                           │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

### **6.7.2 Complexity Summary**

| Data Structure | Operation | Array Impl. | Linked List Impl. |
|----------------|-----------|-------------|-------------------|
| Stack | push | O(1) amortized | O(1) |
| Stack | pop | O(1) amortized | O(1) |
| Queue | enqueue | O(1) amortized | O(1) |
| Queue | dequeue | O(1) amortized | O(1) |
| Deque | push/pop at either end | O(1) amortized | O(1) |
| Priority Queue | insert | O(log n) | O(log n) |
| Priority Queue | extract | O(log n) | O(log n) |

---

## **6.8 Practice Problems**

### **Problem 1: Valid Parentheses**
Given a string containing just the characters '(', ')', '{', '}', '[' and ']', determine if the input string is valid.

### **Problem 2: Min Stack**
Design a stack that supports push, pop, top, and retrieving the minimum element in constant time.

### **Problem 3: Implement Queue using Stacks**
Implement a first-in-first-out (FIFO) queue using only two stacks.

### **Problem 4: Sliding Window Maximum**
Given an array nums and integer k, return the maximum value in each sliding window of size k (use monotonic deque).

### **Problem 5: Evaluate Reverse Polish Notation**
Evaluate the value of an arithmetic expression in Reverse Polish Notation (postfix).

---

## **6.9 Further Reading**

1. **The Art of Computer Programming, Vol 1** by Knuth - Fundamental algorithms including stacks and queues
2. **Introduction to Algorithms (CLRS)** - Stack and queue implementations and applications
3. **Programming Pearls** by Jon Bentley - Algorithm design using stacks
4. **Linux Kernel Source** - Real-world implementations of circular buffers and work queues

---

> **Coming in Chapter 7**: We'll explore **Hash Tables**, one of the most practically important data structures. You'll learn about hash functions, collision resolution strategies (chaining vs open addressing), load factors, and advanced topics like consistent hashing for distributed systems and perfect hashing.

---

**End of Chapter 6**

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='5. linked_lists.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='7. hash_tables.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
